In [1]:
import os
import cv2
import numpy as np

In [2]:
# import torch
# import torch.nn as nn
# from torchvision.models import mobilenet_v2
# from torch.nn import functional as F

# class Eye_model(nn.Module):
#     def __init__(self, num_classes=2):
#         super(Eye_model, self).__init__()

#         # Load pretrained MobilenetV2 model
#         self.base_model = mobilenet_v2(pretrained=True)
#         out_features = self.base_model.classifier[1].out_features
        
#         # Add custom layers
#         self.head = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(out_features, 64),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(64, num_classes),
#         )
        
#         # Freeze base model layers
#         for param in self.base_model.parameters():
#             param.requires_grad = False

#     def forward(self, inp):
#         x = self.base_model(inp)
#         if isinstance(x, tuple):
#             x = x[0]
#         return self.head(x)

# model = Eye_model(num_classes=2)

In [3]:
from model_architecture import Eye_model

model = Eye_model(num_classes=2)

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import torch

# Load weights
model.load_state_dict(torch.load(r"D:\CSES\Year2\IoT\Project\checkpoint.pt", map_location=torch.device('cpu')))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18316\3913100349.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\CSES\Year2\IoT\Project\

<All keys matched successfully>

In [21]:
import cv2
import time
import torch
import numpy as np
from PIL import Image
from collections import deque
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms

def preprocess(img):
  transform = transforms.Compose([
      transforms.Resize(256),
      transforms.RandomCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(10),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  return transform(img)

def display_output(img, predicted_class, landmarks_list=None):
  height, width, _ = img.shape
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 2
  thickness = 3
  color = (0, 0, 255)  # Red

  text = str(predicted_class)
  (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)

  # Bottom-right corner position (10 pixels padding)
  x = width - text_width - 10
  y = height - 10
  cv2.putText(img, text, (x, y), font, font_scale, color, thickness)

  # Draw facial landmarks if available
  if landmarks_list:
      for landmarks in landmarks_list:
          for (x, y) in landmarks:
              cv2.circle(img, (x, y), 2, (0, 255, 0), -1)

  cv2.imshow("output", img)
  return img

def save_image(img, pred_class, count, f_name):
  if pred_class is not None:
    cv2.imwrite('out/{}_frame-{:03d}.jpg'.format(pred_class, count), img)
  else:
    cv2.imwrite('{}/frame-{:03d}.jpg'.format(f_name, count), img)
  count += 1
  return count

def define_labels():
  labels = {
      0 : 'aware',
      1 : 'drowsy',
      
    }
  return labels

def recognize():
    labels = define_labels()
    cam = cv2.VideoCapture(0)
    out_count = 1
    thresh_count = 1
    inp_count = 1
    history = deque(maxlen=7)  # Sliding window of last 7 predictions

    model.eval()
    with torch.no_grad():
        while True:
            ret, img = cam.read()
            if not ret:
                break
            img = cv2.flip(img, 1)
            img_copy = img.copy()

            img = Image.fromarray(img)
            img_tensor = preprocess(img)
            img_tensor = img_tensor.unsqueeze(0)

            pred = model(img_tensor)
            class_id = pred.argmax().item()
            history.append(class_id)

            # Apply majority vote
            stable_pred = max(set(history), key=history.count)
            predicted_class = labels[stable_pred]

            ret_img = display_output(img_copy, predicted_class)

            if cv2.waitKey(1) == ord('s'):
                inp_count = save_image(img_copy, None, inp_count, 'inp')
                out_count = save_image(ret_img, predicted_class, out_count, 'out')
            if cv2.waitKey(1) == ord('q'):
                break

    cam.release()
    cv2.destroyAllWindows()

recognize()